In [1]:
from nilearn import plotting
import pylab as plt
import os
import numpy as np
import nibabel as nb
from scipy.ndimage import affine_transform
from scipy.ndimage import rotate
import albumentations as albu
from albumentations.core.transforms_interface import ImageOnlyTransform, DualTransform
from scipy.stats import multivariate_normal
import cv2
#from cerebrum.config import Config

/Users/alex/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
def addWeighted(src1, alpha, src2, beta, gamma):
    """ 
    Calculates the weighted sum of two arrays (cv2 replaced).

    :param src1: first input array.
    :param aplha: weight of the first array elements.
    :param src2: second input array of the same size and channel number as src1.
    :param beta: weight of the second array elements.
    :param gamma: scalar added to each sum
    :return: output array that has the same size and number of channels as the input arrays.
    """

    return src1 * alpha + src2 * beta + gamma

In [3]:
def augmentation_salt_and_pepper_noise(X_data, amount=10. / 1000):
    """ 
    Function to add S&P noise to the volume.

    :param X_data: input volume (3D) -> shape (x,y,z)
    :param amount: quantity of voxels affected
    :return X_data_out: augmented volume
    """

    X_data_out = X_data
    salt_vs_pepper = 0.2  # Ration between salt and pepper voxels
    n_salt_voxels = int(np.ceil(amount * np.prod(X_data_out.size) * salt_vs_pepper))
    n_pepper_voxels = int(np.ceil(amount * np.prod(X_data_out.size) * (1.0 - salt_vs_pepper)))

    # Add Salt noise
    coords = [np.random.randint(0, i - 1, int(n_salt_voxels)) for i in np.squeeze(X_data_out).shape]
    X_data_out[coords[0], coords[1], coords[2]] = np.max(X_data)

    # Add Pepper noise
    coords = [np.random.randint(0, i - 1, int(n_pepper_voxels)) for i in np.squeeze(X_data_out).shape]
    X_data_out[coords[0], coords[1], coords[2]] = np.min(X_data)

    return X_data_out

In [4]:
def augmentation_gaussian_noise(X_data):
    """ 
    Function to add gaussian noise to the volume.

    :param X_data: input volume (3D) -> shape (x,y,z)
    :return X_data_out: augmented volume
    """

    # Gaussian distribution parameters
    X_data_no_background = X_data
    mean = np.mean(X_data_no_background)
    var = np.var(X_data_no_background)
    sigma = var ** 0.5

    gaussian = np.random.normal(mean, sigma, X_data.shape).astype(X_data.dtype)

    # Compose the output (src1, alpha, src2, beta, gamma)
    X_data_out = addWeighted(X_data, 0.8, gaussian, 0.2, 0)

    return X_data_out

In [5]:
def augmentation_inhomogeneity_noise(X_data, inhom_vol):
    """ 
    Function to add inhomogeneity noise to the volume.

    :param X_data: input volume (3D) -> shape (x,y,z)
    :param inhom_vol: inhomogeneity volume (preloaded)
    :return X_data_out: augmented volume
    """

    # Randomly select a vol of the same shape of 'X_data'
    x_1 = np.random.randint(0, int(X_data.shape[0]) - 1, size=1)[0]
    x_2 = np.random.randint(0, int(X_data.shape[1]) - 1, size=1)[0]
    x_3 = np.random.randint(0, int(X_data.shape[2]) - 1, size=1)[0]
    y_1 = inhom_vol[x_1: x_1 + X_data.shape[0],
          x_2: x_2 + X_data.shape[1],
          x_3: x_3 + X_data.shape[2]]

    # Compose the output: add noise to the original vol
    X_data_out = X_data + y_1.astype(X_data.dtype)

    return X_data_out

In [6]:
def translate_volume(image,
                     shift_x0: int, shift_x1: int, shift_x2: int,
                     padding_mode: str = 'nearest',
                     spline_interp_order: int = 1):
    """ 
    Function to apply volume translation to a single volume.

    :param image: input volume (3D) -> shape (x,y,z)
    :param shift_x0-shift_x1-shift_x2: shift in voxels
    :param padding_mode: the padding mode
    :param spline_interp_order: order for the affine transformation
    :return: augmented volume
    """

    # Set the affine transformation matrix
    M_t = np.eye(4)
    M_t[:-1, -1] = np.array([-shift_x0, -shift_x1, -shift_x2])

    return affine_transform(image, M_t,
                            order=spline_interp_order,
                            mode=padding_mode,
                            cval=0,
                            output_shape=image.shape)


In [7]:
def TranslationAugment(img, shift_x0: int = 0, shift_x1: int = 0, shift_x2: int = 0):
    """ Function to deal with translation augmentation. """
        # Apply to image or mask
    if np.issubdtype(img.dtype, np.floating):  # image
        img_out = translate_volume(img,
                                   shift_x0, shift_x1, shift_x2,
                                   padding_mode='nearest',
                                   spline_interp_order=1)
    elif np.issubdtype(img.dtype, np.integer):  # mask
        img_out = translate_volume(img,
                                   shift_x0, shift_x1, shift_x2,
                                   padding_mode='constant',
                                   spline_interp_order=0)
    else:
        raise Exception('Error 23: type not supported.')

    return img_out

In [8]:
def RotationAugment(img,max_angle: int = 10, rot_spline_order: int = 3,always_apply=False,p=1.0):
        """ Function to deal with rotation augmentation. """
        random_angle = np.random.RandomState().randint(2 * max_angle) - max_angle
        rot_axes = np.random.RandomState().permutation(range(3))[:2]  # random select the 2 rotation axes
        # Apply to image or mask
        if np.issubdtype(img.dtype, np.floating):  # image
            img_out = rotate(input=img,
                             angle=random_angle,
                             axes=rot_axes,
                             reshape=False,
                             order=rot_spline_order,
                             mode='nearest',
                             prefilter=True)
        elif np.issubdtype(img.dtype, np.integer):  # mask
            img_out = rotate(input=img,
                             angle=random_angle,
                             axes=rot_axes,
                             reshape=False,
                             order=0,
                             mode='constant',
                             prefilter=True)
        else:
            raise Exception('Error 24: type not supported.')

        return img_out

In [9]:
def GhostingAugment(img,max_repetitions: int = 4, always_apply=False, p=1.0):
    """Function to deal with ghosting augmentation. """
        # Randomly select parameters
    repetitions = np.random.RandomState().choice(range(1, max_repetitions + 1))
    axis = np.random.RandomState().choice(range(len(img.shape)))

    img_out = img
    shift_value = 0
    for i_rep in range(1, repetitions + 1):
        # Compute the shift to apply to the data
        shift_value += int(img.shape[axis] / (i_rep + 1))

        # Shift the data and add to the out volume
        data_repetition = np.roll(img, shift_value, axis=axis)
        img_out = addWeighted(img_out, 0.85, data_repetition, 0.15, 0)

    return img_out

In [10]:
def augmentation_inhomogeneity_noise(X_data, inhom_vol):
    """ 
    Function to add inhomogeneity noise to the volume.

    :param X_data: input volume (3D) -> shape (x,y,z)
    :param inhom_vol: inhomogeneity volume (preloaded)
    :return X_data_out: augmented volume
    """
    # Randomly select a vol of the same shape of 'X_data'
    x_1 = np.random.randint(0, int(X_data.shape[0]) - 1, size=1)[0]
    x_2 = np.random.randint(0, int(X_data.shape[1]) - 1, size=1)[0]
    x_3 = np.random.randint(0, int(X_data.shape[2]) - 1, size=1)[0]
    y_1 = inhom_vol[x_1: x_1 + X_data.shape[0],
          x_2: x_2 + X_data.shape[1],
          x_3: x_3 + X_data.shape[2]]
    # Compose the output: add noise to the original vol
    X_data_out = X_data + y_1.astype(X_data.dtype)

    return X_data_out


In [11]:
def get_augm_transforms(inho_vol, volume_size: int = 128):
    """
    Get the transformations for volume (and mask) augmentation.

    :param inho_vol: inhomogeneity volume
    :param volume_size: size of the volume
    :param config: Config class (with all probabilities stored)
    :return: albumentation composition
    """

    return albu.Compose([

        # Default transformations
        albu.VerticalFlip(p = 0.8),  # sagittal plane
        InhomogeneityNoiseAugment(inho_vol, p = 0.8),  # Inhomogeneity noise

        # Geometric transformations
        albu.OneOf([
            albu.GridDistortion(num_steps = 5,
                                distort_limit = (-0.10, +0.10),
                                interpolation = 4,
                                border_mode = 1,
                                p = 0.4),
            albu.RandomResizedCrop(height = volume_size,
                                   width = volume_size,
                                   scale = (0.9, 1.0),
                                   ratio = (0.8, 1.20),
                                   interpolation = 4,
                                   p = 0.3),
            RotationAugment(p = 0.5),
            TranslationAugment(p = 0.2),
        ], p = 1.0),

        # Color transformations
        albu.OneOf([
            albu.Blur(blur_limit = (3, 3), p = 0.4),
            albu.Downscale(scale_min = 0.6, 
                           scale_max = 0.99, 
                           interpolation = 4, 
                           p = 0.5),
            SaltAndPepperNoiseAugment(p = 0.4),
            GaussianNoiseAugment(p = 0.8),
            GhostingAugment(p = 0.5),
        ], p = 0.8),

    ], p = 1.0)

In [12]:
dir = '/Users/alex/Documents/MATLAB/data/shepp-logan-dataset/'
basename = 'shepp_logan_'
fileext  = '.nii.gz'
img = []
img_mask = []
for id in range(0,9): 
    fileid = '%05d'%id
    IDdirpath = dir + fileid;
    filename = IDdirpath + '/'+ basename + fileid + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '_mask'+ fileext
    img_name = nb.load(filename)
    img_m = nb.load(maskfilename)
    img.append(img_name)
    img_mask.append(img_m)
 

In [13]:
def inhom_vol(img):
    molt_factor = 2
    max_value = 0.5                     # max (or min) = 0.25 (-0.25) 
    t1 = img #[:,:,:,0]
    # create axis
    x_1 = np.linspace(0, 1, int(t1.shape[0]*molt_factor), endpoint=True)
    x_2 = np.linspace(0, 1, int(t1.shape[1]*molt_factor), endpoint=True)
    x_3 = np.linspace(0, 1, int(t1.shape[2]*molt_factor), endpoint=True)
    x_1,x_2,x_3 = np.meshgrid(x_1,x_2,x_3)
    pos = np.stack((x_1, x_2, x_3),axis=-1)
    # create distrs
    mean = [0.5, 0.5, 0.5]
    cov = [[2.0, 0.3, 0.3], [0.3, 2.0, 0.3], [0.3, 0.3, 2.0]]
    rv = multivariate_normal(mean, cov, allow_singular=True)
    y = rv.pdf(pos)             # derive distribution
    y -= np.mean(y)             # standardisation (I want across zero)
    y = y / max(y.max(), np.abs(y.min())) * max_value
    y = np.transpose(y, (1, 0, 2))      # swap axes
    return y

In [50]:
#Salt & Pepper Noise
for id in range(0,9):
    fileid = '%05d'%id
    IDdirpath = dir + fileid;
    
    vol_out = np.copy(img[id].get_fdata())
    vol_out_m = np.copy(img_mask[id].get_fdata())

    #################### Salt & Pepper Noise ################################## 
    vol_s_p = augmentation_salt_and_pepper_noise(vol_out)
    vol_s_p_m = augmentation_salt_and_pepper_noise(vol_out_m)
    vol_s_p = nb.Nifti1Image(vol_s_p,affine = img[id].header.get_sform(), header = img[id].header)
    vol_s_p_m = nb.Nifti1Image(vol_s_p_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header)
    filename = IDdirpath + '/'+ basename + fileid + '_S&P' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '_S&P_mask'+ fileext    
    nb.save(vol_s_p,filename) #Salt & Pepper Noise
    nb.save(vol_s_p_m,maskfilename) #Salt & Pepper Noise
    print('img%d with Salt & Pepper Noise.....Saved'%id) #Salt & Pepper Noise

    #################### Gaussian Noise ################################## 
    vol_out_gn = augmentation_gaussian_noise(vol_out)
    vol_out_gn_m = augmentation_gaussian_noise(vol_out_m)    
    vol_out_gn = nb.Nifti1Image(vol_out_gn,affine = img[id].header.get_sform(), header = img[id].header)
    vol_out_gn_m = nb.Nifti1Image(vol_out_gn_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header)
    filename = IDdirpath + '/'+ basename + fileid + '_GN' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '_GN_mask'+ fileext    
    nb.save(vol_out_gn,filename) #Gaussian Noise 
    nb.save(vol_out_gn_m,maskfilename) #Gaussian Noise 
    print('img%d with Gaussian Noise.....Saved'%id) #Gaussian Noise  
    
    #################### Translation ##################################    
    vol_out_trans = TranslationAugment(vol_out,shift_x0 = 0,shift_x1 = 30,shift_x2 = 0)
    vol_out_trans_m = TranslationAugment(vol_out_m,shift_x0 = 0,shift_x1 = 30,shift_x2 = 0)
    vol_out_trans = nb.Nifti1Image(vol_out_trans,affine = img[id].header.get_sform(), header = img[id].header)
    vol_out_trans_m = nb.Nifti1Image(vol_out_trans_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header)    
    filename = IDdirpath + '/'+ basename + fileid + '_Trans' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '__Trans_mask'+ fileext    
    nb.save(vol_out_trans,filename) #Translation  
    nb.save(vol_out_trans_m,maskfilename) #Translation
    print('Translated img%d .....Saved'%id) #Translation 
    
    #################### Rotation ##################################
    vol_out_rot = RotationAugment(vol_out)
    vol_out_rot_m = RotationAugment(vol_out_m)
    vol_out_rot = nb.Nifti1Image(vol_out_rot,affine = img[id].header.get_sform(), header = img[id].header)
    vol_out_rot_m = nb.Nifti1Image(vol_out_rot_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header) 
    filename = IDdirpath + '/'+ basename + fileid + '_Rot' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '__Rot_mask'+ fileext    
    nb.save(vol_out_rot,filename) #Rotation  
    nb.save(vol_out_rot_m,maskfilename) #Rotation
    print('Rotated img%d .....Saved'%id) #Rotation 
    
    #################### Ghosting ##################################
    vol_out_gho = GhostingAugment(vol_out)
    vol_out_gho_m = GhostingAugment(vol_out_m)
    vol_out_gho = nb.Nifti1Image(vol_out_gho,affine = img[id].header.get_sform(), header = img[id].header)
    vol_out_gho_m = nb.Nifti1Image(vol_out_gho_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header) 
    filename = IDdirpath + '/'+ basename + fileid + '_Gho' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '__Gho_mask'+ fileext    
    nb.save(vol_out_gho,filename) #Ghosting  
    nb.save(vol_out_gho_m,maskfilename) #Ghosting
    print('Ghosted img%d .....Saved'%id) #Ghosting 
      

    ############################ Inhomogeneity_noise ###################################
    y = inhom_vol(vol_out)
    y_m = inhom_vol(vol_out_m)
    vol_out_inhom = augmentation_inhomogeneity_noise(vol_out,y)
    vol_out_inhom_m = augmentation_inhomogeneity_noise(vol_out_m,y_m)
    vol_out_inhom = nb.Nifti1Image(vol_out_inhom,affine = img[id].header.get_sform(), header = img[id].header)
    vol_out_inhom_m = nb.Nifti1Image(vol_out_inhom_m,affine = img_mask[id].header.get_sform(), header = img_mask[id].header) 
    filename = IDdirpath + '/'+ basename + fileid + '_InhoN' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '__InhoN_mask'+ fileext    
    nb.save(vol_out_inhom,filename) #Inhomogeniety  
    nb.save(vol_out_inhom_m,maskfilename) #Inhomogeniety
    print('img%d with Inhomogeniety noise.....Saved'%id) #Inhomogeniety 

print('Done!')    

img0 with Salt & Pepper Noise.....Saved
img0 with Gaussian Noise.....Saved
Translated img0 .....Saved
Rotated img0 .....Saved
Ghosted img0 .....Saved
img0 with Inhomogeniety noise.....Saved
img1 with Salt & Pepper Noise.....Saved
img1 with Gaussian Noise.....Saved
Translated img1 .....Saved
Rotated img1 .....Saved
Ghosted img1 .....Saved
img1 with Inhomogeniety noise.....Saved
img2 with Salt & Pepper Noise.....Saved
img2 with Gaussian Noise.....Saved
Translated img2 .....Saved
Rotated img2 .....Saved
Ghosted img2 .....Saved
img2 with Inhomogeniety noise.....Saved
img3 with Salt & Pepper Noise.....Saved
img3 with Gaussian Noise.....Saved
Translated img3 .....Saved
Rotated img3 .....Saved
Ghosted img3 .....Saved
img3 with Inhomogeniety noise.....Saved
img4 with Salt & Pepper Noise.....Saved
img4 with Gaussian Noise.....Saved
Translated img4 .....Saved
Rotated img4 .....Saved
Ghosted img4 .....Saved
img4 with Inhomogeniety noise.....Saved
img5 with Salt & Pepper Noise.....Saved
img5 with 

In [71]:
transform = albu.OneOf([
    #albu.GridDistortion(num_steps = 5,distort_limit = (-0.10, +0.10),interpolation = 4,border_mode = 1,p = 1),    
    # Default transformations
    #albu.VerticalFlip(p = 1.),  # sagittal plane
    #albu.RandomResizedCrop(height = 45, width = 67, scale = (0.9, 0.9), ratio = (0.8, 1.20), interpolation = 4,  p = 1),
    #albu.Blur(always_apply = False, p=1.0, blur_limit = (3,3)),
    #albu.CLAHE(always_apply=False, p=1.0, clip_limit = (1,2), tile_grid_size=(8,8)),
    #albu.Downscale(always_apply = False, p=1.0, scale_min=0.6, interpolation=4),
    #albu.ImageCompression(always_apply=False, p=1.0, quality_lower=90, quality_upper=100,compression_type=0),
    #albu.RandomGamma(always_apply=False, p=1.0,gamma_limit=(101,101),eps=1e-07),
    #albu.CenterCrop(always_apply=False, p=1.0,height=100,width=100),
    #albu.HorizontalFlip(always_apply=False, p=1.0),
    albu.ShiftScaleRotate(always_apply=False, p=1.0,shift_limit=(+0.05,0.05), scale_limit=(+0.1,0.1),
                          rotate_limit=(+5,5),interpolation=4, border_mode=1, value=(0,0,0), mask_value=None),

    
], p=1)

In [72]:
for id in range(0,9):
    fileid = '%05d'%id
    IDdirpath = dir + fileid;
    
    vol_out = np.copy(img[id].get_fdata())
    vol_out_m = np.copy(img_mask[id].get_fdata())
    transformed = transform(image = vol_out)
    transformed_m = transform(image = vol_out_m)
    transformed_image = transformed['image']
    transformed_Mask = transformed_m['image']
    transformed_image = nb.Nifti1Image(transformed_image, affine = img[id].header.get_sform(), header = img[id].header)
    transformed_Mask = nb.Nifti1Image(transformed_Mask, affine = img_mask[id].header.get_sform(), header = img_mask[id].header)

    #filename = IDdirpath + '/'+ basename + fileid + '_HorF' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___HorF_mask'+ fileext 
    
    #filename = IDdirpath + '/'+ basename + fileid + '_GridDistortion' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___GridDistortion_mask'+ fileext 
    
    #filename = IDdirpath + '/'+ basename + fileid + '_VerF' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___VerF_mask'+ fileext 
    
    #filename = IDdirpath + '/'+ basename + fileid + '_Blur' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___Blur_mask'+ fileext
    
    #filename = IDdirpath + '/'+ basename + fileid + '_RandomGamma' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___RandomGamma_mask'+ fileext  
    
    #filename = IDdirpath + '/'+ basename + fileid + '_CentCrop' + fileext
    #maskfilename = IDdirpath + '/' + basename + fileid + '___CentCrop_mask'+ fileext  

    filename = IDdirpath + '/'+ basename + fileid + '_ShiScaRot' + fileext
    maskfilename = IDdirpath + '/' + basename + fileid + '___ShiScaRot_mask'+ fileext 
    
    #name = 'Horizontal Flip'
    #name = 'Grid Distortion'
    #name = 'Virtical Flip'
    #name = 'Blurred'
    #name = 'RandomGamma'
    #name = 'Center Crop'
    name = 'Shift Scale Rotate '
    
    nb.save(vol_out_inhom,filename) # HorF -> GridDistortion -> Virtical Flip -> Blurred ->RandomGamma ->Center Crop->ShiScaRot
    nb.save(vol_out_inhom_m,maskfilename) #
    print('img%d with %s .....Saved'%(id,name)) # 
print ('All done!')

img0 with Shift Scale Rotate  .....Saved
img1 with Shift Scale Rotate  .....Saved
img2 with Shift Scale Rotate  .....Saved
img3 with Shift Scale Rotate  .....Saved
img4 with Shift Scale Rotate  .....Saved
img5 with Shift Scale Rotate  .....Saved
img6 with Shift Scale Rotate  .....Saved
img7 with Shift Scale Rotate  .....Saved
img8 with Shift Scale Rotate  .....Saved
All done!
